In [2]:
import pandas as pd

from pandas import DataFrame as df
import numpy as np
import csv
import pickle


In [3]:
otu_animal_sur = pd.read_csv('data/silva/split/Animal_surface.txt', sep = '\t', index_col = 0)
otu_plant_corpus = pd.read_csv('data/silva/split/Plant_corpus.txt', sep = '\t', index_col = 0)
otu_sediment_nonsaline = pd.read_csv('data/silva/split/Sediment_(non-saline).txt', sep = '\t', index_col = 0)
otu_water_saline = pd.read_csv('data/silva/split/Water_(saline).txt', sep = '\t', index_col = 0)



In [12]:
otu_plant_surface = pd.read_csv('data/silva/split/Plant_surface.txt', sep = '\t', index_col = 0)
otu_sediment_saline = pd.read_csv('data/silva/split/Sediment_(saline).txt', sep = '\t', index_col = 0)
otu_surface_nonsaline = pd.read_csv('data/silva/split/Surface_(non-saline).txt', sep = '\t', index_col = 0)
otu_soil_nonsaline = pd.read_csv('data/silva/split/Soil_(non-saline).txt', sep = '\t', index_col = 0)

In [5]:
y = ['animal_dg'] * otu_animal_sur.shape[1] #number of samples
y = y + ['plant_corpus'] * otu_plant_corpus.shape[1]
y = y + ['sed_nosalt'] * otu_sediment_nonsaline.shape[1]
y = y + ['water_salt'] * otu_water_saline.shape[1]

In [15]:
def ratio(shape):
    return(shape[1]/ shape[0])

print(ratio(otu_animal_sur.shape))
print(ratio(otu_plant_corpus.shape))
print(ratio(otu_sediment_nonsaline.shape))
print(ratio(otu_water_saline.shape))

print(ratio(otu_plant_surface.shape))
print(ratio(otu_sediment_saline.shape))
print(ratio(otu_surface_nonsaline.shape))
print(ratio(otu_soil_nonsaline.shape))
print(len(y))

#otu_animal_sur
#plant_surface
#soil_nonsaline



0.048033758304902134
0.009912919656060025
0.011569979056101182
0.014144743295470151
0.04105058831987347
0.01013953488372093
0.021521701701388344
0.0547954932777883
5216


In [33]:
#Throw samples with less than 5000 reads
sample_sums = np.sum(otu, axis = 0)

In [37]:
otu_filter = otu.loc[:, sample_sums > 5000]
print(otu_filter.shape)

(35511, 15475)


In [38]:
taxa_sums = np.sum(otu_filter, axis = 1)
otu_filter = otu_filter.loc[taxa_sums > 0, :]
print(otu_filter.shape)

(35275, 15475)


In [4]:
#Only useful for reducing data size when using already subsetted data. If using whole dataset, all taxa are present at least once
#total_taxa_abund = otu.sum(axis = 1) #Should be ntaxa (5000) of them
#otu_prune = otu.loc[total_taxa_abund > 0, :]
#print("After deleting taxa that never appear, we have " + str(otu_prune.shape[0]) + " taxa in " + str(otu_prune.shape[1]) + " samples")

In [42]:
thresh = 0.0002 * otu_filter.shape[1] # at least 1 % of samples (10 in this case)
print("Only keep taxa present in greater than " + str(thresh) + " samples")
binary = otu_filter > 0
keep = binary.sum(axis = 1) >= thresh# Should be ntaxa
binary = binary.loc[keep, :]
print("We will keep " + str(keep.sum()) +" taxa")


Only keep taxa present in greater than 3.095 samples
We will keep 21979 taxa


In [52]:
print(len(keep))
print(binary.shape)

35275
(21979, 15475)


In [50]:
#Split out a test set

np.random.seed(15)
test_samples = np.random.choice(binary.columns.values, 1500)
f = open("data/AG_new/AG_test_samples.obj", "wb")
pickle.dump(test_samples, f)
f.close()

binary_train = binary.loc[:, [not(i in test_samples) for i in binary.columns.values ]]
otu_train = otu_filter.loc[:, [not(i in test_samples) for i in binary.columns.values ]]
otu_test = otu_filter[test_samples]


In [53]:
#Test text output with small file
#otu_filter["#OTU ID"].head()
#tmp = otu_filter.head()
#tmp = tmp.iloc[:, [0,1,2,3,4]]
#tmp.head()
#tmp.to_csv("silva/test_out.csv", sep = "\t", index = False)
print(otu_train.shape)
print(otu_test.shape)
print(otu_filter.shape)

(35275, 14027)
(35275, 1500)
(35275, 15475)


In [54]:
otu_train_filter = otu_train.loc[np.array(keep), :]
otu_test_filter = otu_test.loc[np.array(keep), :]
otu_filter = otu_filter.loc[keep, :]

otu_filter.to_csv("data/AG_new/otu_filtered_AG_02perc.csv", sep = "\t", index = True)
otu_train_filter.to_csv("data/AG_new/otu_filtered_train_AG_02perc.csv", sep = "\t", index = True)
otu_test_filter.to_csv("data/AG_new/otu_filtered_test_AG_02perc.csv", sep = "\t", index = True)

In [55]:
print(otu_filter.shape)

(21979, 15475)


In [56]:
file_text = []
for i in range(binary_train.shape[1]):
    taxa_share = binary_train.index[binary_train.iloc[:,i] == True]
    file_text.append(taxa_share)

In [57]:
lengths = []
for taxa_list in file_text:
    lengths.append(len(taxa_list))
max_val = max(lengths)
max_ind = np.argmax(lengths)
print("Max number of taxa in any given sample is: " + str(max_val))
print("Found at " + str(max_ind))


Max number of taxa in any given sample is: 3590
Found at 43


In [58]:
#Write Glove input file
with open('data/AG_new/glove_input_AG_02perc.txt', mode = 'w', newline='') as file:
    writer = csv.writer(file, delimiter = "\t", quoting = csv.QUOTE_NONE, escapechar = '')
    for taxa_list in file_text:
        writer.writerow(taxa_list)
file.close()
